In [22]:
import pandas as pd

In [62]:
# data_path = './data/data_1mb.csv'
# data_path = './data/data_1gb.csv'
data_path = './data/data_10gb.csv'
# data_path = './data/data_30gb.csv'

In [63]:
df = pd.read_csv(data_path)

In [64]:
df.head()

,user_id,timestamp,amount,currency,card_number,merchant,country,is_fraud
0,6,2025-03-17T23:19:22.556261,1458.76,USD,6539162721261779,"Thomas, Allen and Griffin",Philippines,False
1,243,2025-05-07T08:14:32.184651,514.10,USD,4311622618139486,Carlson-White,Papua New Guinea,False
2,79,2025-02-14T03:23:13.206927,1471.02,USD,375417705000324,Berg Inc,Seychelles,False
3,50,2025-01-08T20:39:46.516680,3076.68,USD,4578901718125613,Scott Ltd,Mauritania,False
4,706,2025-05-10T15:23:00.456036,1101.41,USD,4927482106655881,Montoya Ltd,Faroe Islands,False


In [65]:
df.describe()

,user_id,amount,card_number
count,1.206451e+08,1.206451e+08,1.206451e+08
mean,5.004684e+02,2.502454e+03,3.773892e+17
std,2.886853e+02,1.441993e+03,1.246205e+18
min,1.000000e+00,5.000000e+00,6.040000e+10
25%,2.500000e+02,1.253680e+03,3.899863e+13
50%,5.000000e+02,2.502420e+03,3.512511e+15
75%,7.500000e+02,3.751200e+03,4.625468e+15
max,1.000000e+03,5.000000e+03,5.000000e+18


In [66]:
# ile unikalnych userów?
len(df['user_id'].unique())



1000

In [67]:
# popularnosc sprzedawcow
df.groupby("merchant").size().sort_values(ascending=False)

merchant
Williams Ltd              172212
Smith Group               165563
Smith PLC                 164450
Smith LLC                 164037
Smith Ltd                 141550
                           ...  
Henry, Chen and Burns          1
Henry, Chen and Brown          1
Henry, Chen and Briggs         1
Henry, Chen and Bond           1
Lee, Craig and Weeks           1
Length: 27308714, dtype: int64

In [68]:
# fraud detection

# większość transakcji jednego usera jest z jednego kraju, 
# jako fraud bedziemy identyfikowali transakcje ktore są wykonane z innego kraju

user_country_counts = df.groupby(['user_id', 'country']).size().reset_index(name='count')
user_most_common_country = user_country_counts.sort_values('count', ascending=False).groupby('user_id').first()

# dodajemy kolumne z najczesciejszym krajem dla danego usera
df_with_common_country = df.merge(
    user_most_common_country[['country']], 
    on='user_id', 
    suffixes=('', '_most_common')
)

# flagujemy transakcje które są wykonane z innego kraju
df['is_country_anomaly'] = df_with_common_country['country'] != df_with_common_country['country_most_common']

# Show transactions that are potential fraud based on country anomaly
country_anomaly_transactions = df[df['is_country_anomaly']]

In [69]:
country_anomaly_transactions

,user_id,timestamp,amount,currency,card_number,merchant,country,is_fraud,is_country_anomaly
91,276,2025-02-26T04:12:06.093832,4267.20,USD,379654095189236,"Shaw, Gross and Smith",Hungary,True,True
114,997,2025-01-18T19:49:20.107546,1235.01,USD,213101170861446,"Gray, Lopez and Diaz",China,True,True
196,845,2025-05-05T14:59:25.889815,436.80,USD,639017064739,"Chavez, Smith and Whitney",Australia,True,True
351,61,2025-01-26T05:27:08.790318,3358.25,USD,4389224905053061,Lowery-Wilson,Western Sahara,True,True
550,109,2025-01-29T10:36:37.154047,3371.63,USD,4501378789757,Lopez-Christensen,Czech Republic,True,True
...,...,...,...,...,...,...,...,...,...
120644653,785,2025-04-12T16:12:42.929146,1057.88,USD,213155284249245,"Eaton, Harper and Bennett",Belize,True,True
120644722,29,2025-01-03T15:56:27.345241,1914.58,USD,213144784346107,Stephens-Gibson,Algeria,True,True
120644869,229,2025-03-16T10:28:06.480631,2470.91,USD,3596037048821272,Snow-Ortiz,Oman,True,True
120644881,235,2025-02-07T09:08:27.274142,1061.65,USD,4253839017476828,"Hurst, Ortega and Miller",Brazil,True,True


In [70]:
len(country_anomaly_transactions) / len(df) * 100.0

0.9975204307428924